## Day 46 Proyect: Spotify Billboard Playlist

### Proyecto Día 46 – Playlist de Billboard 100 en Spotify 🎶

Este proyecto permite al usuario crear una playlist en Spotify con las 100 canciones más populares de una fecha específica.  
Combina scraping web, autenticación OAuth y llamadas a la API de Spotify mediante Spotipy.

🔧 Tecnologías aplicadas:
- `requests` y `BeautifulSoup` para scraping de Billboard
- `spotipy` para interacción con la API de Spotify
- Manejo de listas, excepciones y autenticación


#### Paso 1 – Scraping del Billboard Hot 100

Solicitamos al usuario una fecha y obtenemos las 100 canciones más populares ese día desde Billboard.

In [1]:
import requests                             # Libreria para hacer peticiones HTTP
from bs4 import BeautifulSoup               # Libreria para parsear/extraer datos de HTML
import spotipy                              # Libreria para interactuar con la API de Spotify
from spotipy.oauth2 import SpotifyOAuth     # Libreria para autenticacion con Spotify
import os                                   # Libreria para interactuar con el sistema operativo
from dotenv import load_dotenv              # Libreria para cargar variables de entorno desde un archivo .env

In [2]:
# Carga las variables del archivo .env
load_dotenv()  

True

In [3]:
# Solicitar al usuario la fecha
fecha_de_interes = input("¿A qué fecha quieres viajar? (YYYY-MM-DD): ")

# Generar la URL de la página de Billboard Hot 100 con la fecha proporcionada
url = f"https://www.billboard.com/charts/hot-100/{fecha_de_interes}"

# Crear un encabezado para la solicitud HTTP (esto es útil para evitar bloqueos por parte del servidor)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36"
}

# Realiza la solicitud GET al endpoint de la página web
response = requests.get(url=url
                    , headers=headers)

# Crea un objeto BeautifulSoup para parsear el HTML
soup = BeautifulSoup(response.text, "html.parser")

In [4]:
# Extraer los títulos de canciones
titulos = [tag.getText().strip() for tag in soup.select("li ul li h3")]     # Lista de comprehensión para extraer los títulos de las canciones

In [5]:
# Imprime el número de títulos extraídos
print(f"Se encontraron {len(titulos)} canciones.\n")

# Imprime los primeros 10 títulos extraídos
print(f"Primeros 10 títulos de películas extraidos:")
for title in titulos[:10]:  
    print(f"- {title}")

Se encontraron 100 canciones.

Primeros 10 títulos de películas extraidos:
- A Bar Song (Tipsy)
- I Had Some Help
- Espresso
- Die With A Smile
- Birds Of A Feather
- Taste
- Good Luck, Babe!
- Please Please Please
- Lose Control
- Not Like Us


#### Paso 2 – Autenticación con Spotify (Spotipy)

Usamos OAuth 2.0 para autorizar la creación de playlists privadas desde nuestra cuenta.


In [6]:
# Datos de autenticación 
SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
SPOTIFY_SECRET = os.getenv("SPOTIFY_SECRET")
SPOTIFY_REDIRECT_URI = os.getenv("SPOTIFY_REDIRECT_URI")

In [7]:
# Crea una instancia de la clase SpotifyOAuth para manejar la autenticación
sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=SPOTIFY_CLIENT_ID,
        client_secret=SPOTIFY_SECRET,
        redirect_uri=SPOTIFY_REDIRECT_URI,
        scope="playlist-modify-private",
        show_dialog=True,
        cache_path="token.txt"
    )
)

# Obtiene el id del usuario autenticado
user_id = sp.current_user()["id"]
print(f"Autenticado como usuario: {user_id}")

Autenticado como usuario: riveraderrick5


#### Paso 3 – Buscar las canciones en Spotify y recolectar URIs

Para cada canción del Billboard buscamos su URI en Spotify. Algunas no estarán disponibles.


In [8]:
# Lista para almacenar los URIs (identificadores únicos) de las canciones encontradas
canciones_uris = []

# Extrae el año de la fecha de interés, asumiendo el formato "YYYY-MM-DD"
año = fecha_de_interes.split("-")[0]

# Itera sobre cada título de canción previamente extraído
for titulo in titulos:
    # Realiza una búsqueda en Spotify por nombre de canción y año
    resultado = sp.search(q=f"track:{titulo} year:{año}"
                        ,type="track"
                        ,limit=1)

    try:
        # Intenta obtener el URI de la primera canción encontrada
        uri = resultado["tracks"]["items"][0]["uri"]
        canciones_uris.append(uri)                              # Agrega el URI a la lista si se encontró
    except IndexError:
        # Si no se encuentra la canción, muestra un mensaje informativo
        print(f"No disponible en Spotify: {titulo}")


In [9]:
print(f"Se encontraron {len(canciones_uris)} canciones en Spotify.")

Se encontraron 100 canciones en Spotify.


#### Paso 4 – Crear una nueva playlist privada y añadir canciones

Creamos una playlist llamada "YYYY-MM-DD Billboard 100" y añadimos los URIs obtenidos.


In [10]:
# Crea una nueva playlist privada en el perfil del usuario autenticado
playlist = sp.user_playlist_create(
    user=user_id,                                                    # ID del usuario autenticado en Spotify
    name=f"{fecha_de_interes} Billboard 100",                        # Nombre dinámico con la fecha seleccionada
    public=False,                                                    # False = playlist privada
    description="Playlist creada automáticamente con Python 💻🎶"   # Descripción opcional
)

# Agrega las canciones encontradas (por sus URIs) a la nueva playlist
sp.playlist_add_items(playlist_id=playlist["id"], items=canciones_uris)

print("✅ Playlist creada exitosamente.")


✅ Playlist creada exitosamente.


#### 🧠 Conclusión

Este proyecto integró scraping, autenticación y uso de APIs para lograr un resultado real:

- 🔍 Web scraping estructurado con BeautifulSoup
- 🔐 Autenticación OAuth con Spotipy
- 🧬 Integración entre dos plataformas (Billboard + Spotify)
- 📃 Manipulación de listas, errores y texto